In [18]:
import yfinance as yf
import pandas as pd

# Fetch data
def fetch_data(ticker):
    # Retrieve stock data
    stock = yf.Ticker(ticker)
    history = stock.history(period="3y", interval="1d",auto_adjust=False)
    data = pd.DataFrame(history)
    data.reset_index(inplace=True)
    data["Date"] = data["Date"].dt.strftime("%Y-%m-%d")
    data["ticker"] = ticker

    history = stock.history(period="3y", interval="1d",auto_adjust=True)
    data_adj = pd.DataFrame(history)
    data_adj.reset_index(inplace=True)
    data_adj["Date"] = data_adj["Date"].dt.strftime("%Y-%m-%d")
    data_adj["ticker"] = ticker

    return data, data_adj

In [58]:
import yfinance as yf
import pandas as pd

# 관심 ETF 리스트
etfs = {"QQQI":"QQQ", "MSTY":"MSTR",
        "TSLY":"TSLA", "AMZY":"AMZN",
        "NFLY":"NFLX", "NVDY":"NVDA"}
etfs_data = []
etfs_adj_data = []
stock_data = []
for etf_key, etf_value in etfs.items():
    data, data_adj = fetch_data(etf_key)
    etfs_data.append(data)
    etfs_adj_data.append(data_adj)

    data, _ = fetch_data(etf_value)
    stock_data.append(data)

etfs_data = pd.concat(etfs_data)
etfs_adj_data = pd.concat(etfs_adj_data)
stock_data = pd.concat(stock_data)

In [59]:
import altair as alt

# Create a 2x2 grid of comparison charts for 'Close' prices of etfs_data and etfs_adj_data
charts = []

for etf_key, etf_value in etfs.items():
    etfs_data_tmp = etfs_data[etfs_data['ticker'] == etf_key].copy()
    etfs_adj_data_tmp = etfs_adj_data[etfs_adj_data['ticker'] == etf_key].copy()
    stock_data_tmp = stock_data[(stock_data.Date >= etfs_data_tmp.Date.min())*(stock_data['ticker'] == etf_value)].copy()

    etfs_data_tmp['Close_nomalized'] = etfs_data_tmp['Close'] / (etfs_data_tmp['Close'].max() - etfs_data_tmp['Close'].min())
    etfs_adj_data_tmp['Close_nomalized'] = etfs_adj_data_tmp['Close'] / (etfs_adj_data_tmp['Close'].max() - etfs_adj_data_tmp['Close'].min())
    stock_data_tmp['Close_nomalized'] = stock_data_tmp['Close'] / (stock_data_tmp['Close'].max() - stock_data_tmp['Close'].min())

    comparison_chart = alt.Chart(etfs_data_tmp).mark_line(color='blue').encode(
        x='Date:T',
        y='Close_nomalized:Q',
        tooltip=['ticker:N', 'Date:T', 'Close:Q'],
        color=alt.value('blue')  # Set color for the legend
    ).properties(
        title=f'{etf_key} Close Price Comparison',
        width=800,
        height=400
    )

    # Add adjusted data to the chart
    adjusted_chart = alt.Chart(etfs_adj_data_tmp).mark_line(color='orange').encode(
        x='Date:T',
        y='Close_nomalized:Q',
        tooltip=['ticker:N', 'Date:T', 'Close:Q'],
        color=alt.value('orange')  # Set color for the legend
    )

    # Add stock data to the chart
    stock_chart = alt.Chart(stock_data_tmp).mark_line(color='red').encode(
        x='Date:T',
        y='Close_nomalized:Q',
        tooltip=['ticker:N', 'Date:T', 'Close:Q'],
        color=alt.value('red')  # Set color for the legend
    )

    # Combine both charts
    final_chart = comparison_chart + adjusted_chart + stock_chart
    charts.append(final_chart)


# Create a 2x2 layout for the charts
grid = alt.vconcat(
    alt.hconcat(*charts[:2]),  # First row
    alt.hconcat(*charts[2:4]),   # Second row
    alt.hconcat(*charts[4:])   # Second row
).resolve_scale(
    y='shared'  # Share the y-axis scale across charts
)

# Display the grid of charts
grid.display()

alt.VConcatChart(...)